# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

Framework 와 library 는 각각 코드를 더 쉬운방법으로 구현하고, 사용할 수 있도록 해주지만 두가지는 차이가 있습니다. 

먼저, library 의 경우에는 제 3자인 developer들이 software를 control 할 수 있도록 만들어 놓은 application이라고 생각할 수 있습니다. 그 것을 다운로드 받아 user들이 쉽게 사용할 수 있습니다. 

Framework는 application이라고 생각하기보다도 잘 프로그램할 수 있도록 짜놓은 기본적인 틀이리고 할 수 있습니다. coder들이 언제든지 코드를 수정할 수 있습니다. 종류로는 스프링, 스트럿츠 등이 있습니다. 

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

사실, 딥러닝과 머신러닝모두 인공지능이라는 개념안에서 묶이는 하위개념이라고 할 수 있습니다. 머신러닝은 통계분석 이후에 조금더 알고리즘에 집중함으로서 빅데이터를 예측하거나 분석하는것에 큰 역할을 주고 있습니다. 

반면 딥러닝은 스스로 데이터를 학습하는 것으로 생각됩니다. 특히 신경망으로 이루어진 구조는 인간의 지는 생물학적 구조를 기반으로 만들었고, 컴퓨터가 스스로 이미지, 텍스트 등 다양한 데이터를 학습할 수 있도록 합니다. 

### Q3. 아래의 코드에 주석 달기.

In [2]:
# Library import 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# device 설정, torch cuda가 사용가능한지 확인할 수 있게해준다. 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

cpu is available


In [4]:

learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [5]:
# train , test split 후 dataset 업로드 후 저장한다. 
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
# test셋도 같은 방법으로 저장한다 
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)


Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw




Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw




Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw




Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



In [6]:
# batch size에 맞도록 data loader를 만든다. 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
# target merged!
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [7]:
#기본 함수를 import 한다. 
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 

# 신경망 첫번째 forward부분 
  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [8]:
# model import 
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# epoch 별로 모델을 작동시킨다. 
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

<ipython-input-7-0da6e7ed6201>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1]  cost = 0.329662144


In [ ]:
# 손실함수를 계산한 후 손실값을 평가하는 코드 
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.